In [2]:
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt

In [3]:
n = 100

In [83]:
A = np.random.randn(n, n)
At = torch.Tensor(A).cuda()

In [5]:
def gen_data(samples: int = 100000) -> (np.array, np.array):
    x = np.random.rand(samples, n)
    y = x @ A
    return x, y

In [29]:
class Model(nn.Module):

    def __init__(self, dim, nlayers):
        super().__init__()At
        self.layers = nn.Sequential(
            *[nn.Linear(dim, dim, bias=False) 
              for _ in range(nlayers)]
        )
    
    def forward(self, x):
        return self.layers(x)

In [86]:
for _ in model.parameters():
    print(_)

Parameter containing:
tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.]], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.]], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.]], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([[1., 0., 0.,  ..., 0., 0., 0.],

In [90]:
model = Model(dim=n, nlayers=5).cuda()

optimizer = torch.optim.SGD(model.parameters(), lr=1e-6, momentum=0.9)
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-6)

bsz = 100
niters = 100000

# Init weights at identity
# with torch.no_grad():
#     for layer in model.layers:
#         layer.weight = nn.Parameter(torch.eye(*layer.weight.shape, device='cuda'))
#     print(torch.allclose(model(xt), xt))

# Training using SGD
model.train()
for i in range(niters):
    xt = torch.randn(bsz, n).cuda()
    yt = xt @ At
    pred = model(xt)
    loss = torch.norm(pred - yt)
    model.zero_grad()
    loss.backward()
    optimizer.step()
    if i % 500 == 0:
        with torch.no_grad():
            W = torch.eye(n).cuda()
            for layer in model.layers:
                W = layer.weight @ W
#             print(W)
            diff = W.T - torch.Tensor(A).cuda()
            print(torch.norm(diff))

tensor(100.5279, device='cuda:0')
tensor(100.5249, device='cuda:0')
tensor(100.5219, device='cuda:0')
tensor(100.5188, device='cuda:0')
tensor(100.5157, device='cuda:0')
tensor(100.5127, device='cuda:0')
tensor(100.5096, device='cuda:0')
tensor(100.5065, device='cuda:0')
tensor(100.5035, device='cuda:0')
tensor(100.5004, device='cuda:0')
tensor(100.4973, device='cuda:0')
tensor(100.4942, device='cuda:0')
tensor(100.4911, device='cuda:0')
tensor(100.4880, device='cuda:0')
tensor(100.4849, device='cuda:0')
tensor(100.4818, device='cuda:0')
tensor(100.4786, device='cuda:0')
tensor(100.4755, device='cuda:0')
tensor(100.4723, device='cuda:0')


KeyboardInterrupt: 

In [78]:
At

tensor([[ 1.1555, -0.9323,  0.3376,  ..., -1.5485,  1.0680,  0.3074],
        [ 0.2144, -1.0693,  0.1969,  ...,  1.0020, -0.5260,  0.0681],
        [-1.0021, -1.2606, -0.1730,  ...,  0.2601,  1.0599, -1.1471],
        ...,
        [-1.5374, -0.6431, -0.0801,  ..., -0.0371,  0.2346,  0.1878],
        [-0.7548, -1.7594, -0.8051,  ...,  0.4508, -0.5758,  0.3078],
        [-0.0044, -1.4124,  0.5957,  ..., -2.5750,  0.5862, -2.2354]],
       device='cuda:0')

In [378]:
xs

array([[0.10932015, 0.71752094, 0.60020477, ..., 0.63661339, 0.82869482,
        0.11435129],
       [0.86037369, 0.80902034, 0.1164953 , ..., 0.91079524, 0.65242823,
        0.95507414],
       [0.87359904, 0.58190939, 0.24361753, ..., 0.34816971, 0.73898337,
        0.04573685],
       ...,
       [0.06459961, 0.44896217, 0.15927479, ..., 0.85528033, 0.35310832,
        0.77684412],
       [0.7557412 , 0.40732111, 0.71869546, ..., 0.20994377, 0.69256793,
        0.42008488],
       [0.61420604, 0.35513015, 0.77741073, ..., 0.88769545, 0.33407008,
        0.39353526]])

In [379]:
pred = model(torch.Tensor(xs))

In [405]:
with torch.no_grad():
    W = torch.eye(n)
    for layer in model.layers:
        W = layer.weight @ W
        
print(W.T)

tensor([[ 1.7992,  0.7655,  0.0963,  0.9617, -0.4760,  0.2271,  0.7020,  1.1787,
          1.7783,  0.0112],
        [ 0.3129, -0.2450, -0.7395, -1.2475, -0.1956, -0.1695,  0.2950,  0.9331,
         -1.0711,  0.0429],
        [-0.8792, -0.9184,  0.2616,  0.8375,  0.4000,  0.9495, -0.2676, -0.6877,
         -0.5796, -2.2028],
        [ 1.3109,  0.6477,  0.5951,  0.0856, -0.8735,  0.3254,  1.1232, -0.5331,
          0.0744,  0.3062],
        [-0.2717, -1.5092, -0.3172,  0.8039,  0.2627, -0.8339,  0.5414, -0.4199,
          0.1151,  0.2581],
        [-1.0379, -0.7972, -1.9211, -1.1194,  1.1981,  1.0703,  0.7406, -0.2351,
         -0.2842, -0.1996],
        [ 1.4370, -0.5936,  3.0019, -1.1861, -0.5165,  0.4741, -1.2977,  0.7966,
         -0.8950, -0.4288],
        [ 1.6625,  0.5320, -1.3471, -1.2799, -0.0579, -1.4638,  0.4341,  0.1112,
         -0.6111, -1.0229],
        [-0.3189,  1.9478,  0.2768, -0.1287,  2.4045, -0.2611, -1.7988, -1.3944,
         -0.3518,  0.1793],
        [-0.1222,  

In [401]:
A

array([[ 1.73524426,  0.7589897 ,  0.11086273,  1.00671102, -0.49302239,
         0.27493598,  0.68534215,  1.1878646 ,  1.82161226,  0.03882611],
       [ 0.35287121, -0.22092156, -0.81304522, -1.31859636, -0.19026202,
        -0.25312937,  0.3098661 ,  0.93826705, -1.13002378,  0.02976269],
       [-0.84191622, -0.93250287,  0.28175968,  0.85076471,  0.429506  ,
         0.9517281 , -0.24435565, -0.71093685, -0.57497804, -2.2132725 ],
       [ 1.29654281,  0.65774245,  0.61975526,  0.08412348, -0.89597071,
         0.3446083 ,  1.09790046, -0.52130836,  0.07378989,  0.30467943],
       [-0.24822845, -1.5107785 , -0.32437893,  0.80328529,  0.29162767,
        -0.83795115,  0.5585567 , -0.43618552,  0.11498035,  0.25189669],
       [-1.03149867, -0.79519772, -1.95261088, -1.13940969,  1.22089415,
         1.04493519,  0.74929207, -0.2435886 , -0.30148036, -0.19979712],
       [ 1.39843014, -0.55150968,  2.93359866, -1.25862434, -0.5757817 ,
         0.4263465 , -1.33717181,  0.84676836

In [ ]:
ax = plt.subplot2grid((2, 3), (0, 1),
                      rowspan=2, colspan=2,
                      projection='3d')
ax.view_init(elev=10, azim=-100)
ax.scatter(xs, zs, ys, s=0.5, c=col / 255.0)
ax.set_xlabel('x')
ax.set_ylabel('z')
ax.set_zlabel('y')
ax.set_xlim(-1, 1)
ax.set_ylim(-1, 1)
ax.set_zlim(-1, 1)
ax.set_box_aspect((1, 1, 1))
ax.set_xticks(ax.get_xticks())
ax.set_yticks(ax.get_yticks()[::2])
ax.set_zticks(ax.get_zticks())
plt.show()